# Data Processing Pipeline

In [1]:
from dataset import NewsData

In [2]:
dataset = NewsData(data_path='data/Eluvio_DS_Challenge.csv', votes_threshold=10)

In [3]:
next(iter(dataset))

('Scores killed in Pakistan clashes', 0)

In [4]:
dataset[509235]

('Palestinian wielding knife shot dead in West Bank: Israel police', 0)

In [5]:
len(dataset)

509236

In [6]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from tqdm import tqdm

tokenizer = get_tokenizer('basic_english')
counter = Counter()
for title, label in tqdm(dataset):
    counter.update(tokenizer(title))
    
vocab = Vocab(counter, min_freq=1)

100%|██████████| 509236/509236 [00:14<00:00, 34430.66it/s]


- Here, the vocabulary block converts a list of tokens into integers

In [9]:
[vocab[token] for token in ['us', 'say', 'here', 'example']]

[27, 108, 973, 3812]

- Prepare the text processing pipeline with the tokenizer and vocabulary 
- The text and label pipelines will be used to process the raw data strings from the dataset iterators

In [10]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

In [11]:
text_pipeline('this is an example')

[94, 13, 33, 3812]

In [15]:
import torch
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [23]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _text, _label in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), label_list.to(device), offsets.to(device)

In [24]:
train_iter = NewsData(data_path='data/Eluvio_DS_Challenge.csv', votes_threshold=10)

In [25]:
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

# Model

In [26]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [27]:
num_class = 2
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [28]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (text, label, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, label, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [29]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = NewsData(data_path='data/Eluvio_DS_Challenge.csv', votes_threshold=10)
train_dataset = list(train_iter)
num_train = int(len(train_dataset) * 0.8)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
#                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 6366 batches | accuracy    0.651
| epoch   1 |  1000/ 6366 batches | accuracy    0.653
| epoch   1 |  1500/ 6366 batches | accuracy    0.653
| epoch   1 |  2000/ 6366 batches | accuracy    0.650
| epoch   1 |  2500/ 6366 batches | accuracy    0.652
| epoch   1 |  3000/ 6366 batches | accuracy    0.655
| epoch   1 |  3500/ 6366 batches | accuracy    0.655
| epoch   1 |  4000/ 6366 batches | accuracy    0.654
| epoch   1 |  4500/ 6366 batches | accuracy    0.655
| epoch   1 |  5000/ 6366 batches | accuracy    0.657
| epoch   1 |  5500/ 6366 batches | accuracy    0.659
| epoch   1 |  6000/ 6366 batches | accuracy    0.663
-----------------------------------------------------------
| end of epoch   1 | time: 25.42s | valid accuracy    0.667 
-----------------------------------------------------------
| epoch   2 |   500/ 6366 batches | accuracy    0.664
| epoch   2 |  1000/ 6366 batches | accuracy    0.655
| epoch   2 |  1500/ 6366 batches | accuracy    0.659
| epoch  